# TF в Colab

In [0]:
%config IPCompleter.greedy=True  # более лучший автокомплит

In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.13.0-rc1


In [3]:
tf.reset_default_graph()
a = tf.placeholder(tf.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
print(c)

Tensor("matmul:0", shape=(2, 2), dtype=float32)


In [5]:
print(list(tf.get_default_graph().get_operations()[-1].inputs))
print(list(tf.get_default_graph().get_operations()[-1].outputs))

[<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>, <tf.Tensor 'Variable/read:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'matmul:0' shape=(2, 2) dtype=float32>]


In [0]:
s = tf.InteractiveSession()

In [12]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

array([[2., 2.],
       [2., 2.]], dtype=float32)

In [0]:
s.close()

# Оптимизация

In [0]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])

In [17]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [18]:
with tf.Session() as s:  # сессия сама закроется
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

-0.5737303 0.5143226
-0.45898423 0.32916644
-0.36718738 0.21066652
-0.2937499 0.13482657
-0.23499992 0.086289
-0.18799993 0.055224966
-0.15039995 0.035343975
-0.12031996 0.022620145
-0.09625597 0.0144768935
-0.077004775 0.009265212


# TensorBoard логирование

In [0]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [0]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [0]:
with tf.Session() as s:
  summary_writer = tf.summary.FileWriter("logs/1", s.graph)
  s.run(tf.global_variables_initializer())
  for i in range(10):
      _, curr_summaries = s.run([step, summaries])
      summary_writer.add_summary(curr_summaries, i)
      summary_writer.flush()

Локально можно запустить  `tensorboard --logdir=./logs` в консоли.

А вот так можно запустить прямо в Google Colab:

In [0]:
! wget https://raw.githubusercontent.com/ZEMUSHKA/skillfactory-dl/master/setup_google_colab.py -O setup_google_colab.py -q
import setup_google_colab

# запускаем сервер tensorboard в фоне
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# открываем порт из Google Colab
setup_google_colab.expose_port_on_colab(6006)

# Обучение линейной модели

In [93]:
# модельные данные
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.58814689 0.78643231 0.79772526]]


In [94]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D), name="features")
target = tf.placeholder(tf.float32, shape=(None, 1), name="target")

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights


loss = tf.reduce_mean((target - predictions) ** 2, name="loss")

print(target.shape, predictions.shape, loss.shape)

(?, 1) (?, 1) ()


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [0]:
# производные это часть графа
tf.get_default_graph().get_operations()

In [36]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss)

1.9895532
0.06145981
0.046104774
0.0433546
0.042860366
0.04277123


In [37]:
# результат обучения
curr_weights.T

array([[0.79846495, 0.57642007, 0.56647074]], dtype=float32)

In [38]:
# правильные веса
w.T

array([[0.7692187 , 0.59123648, 0.56121469]])

# Обучение в Keras

In [41]:
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
print(keras.__version__)

2.2.4


In [85]:
K.clear_session()  # очищает граф и перезапускает сессию

model = M.Sequential()
model.add(L.Dense(1, use_bias=False, input_shape=(D,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

In [87]:
model.fit(x, y, batch_size=8, epochs=10, callbacks=[keras.callbacks.TensorBoard("./logs/keras")])

Epoch 1/10
1000/1000 [==============================] - 0s 121us/step - loss: 1.4677 - mean_squared_error: 1.4677
Epoch 2/10
1000/1000 [==============================] - 0s 90us/step - loss: 0.0819 - mean_squared_error: 0.0819
Epoch 3/10
1000/1000 [==============================] - 0s 92us/step - loss: 0.0521 - mean_squared_error: 0.0521
Epoch 4/10
1000/1000 [==============================] - 0s 88us/step - loss: 0.0488 - mean_squared_error: 0.0488
Epoch 5/10
1000/1000 [==============================] - 0s 113us/step - loss: 0.0467 - mean_squared_error: 0.0467
Epoch 6/10
1000/1000 [==============================] - 0s 87us/step - loss: 0.0454 - mean_squared_error: 0.0454
Epoch 7/10
1000/1000 [==============================] - 0s 91us/step - loss: 0.0445 - mean_squared_error: 0.0445
Epoch 8/10
1000/1000 [==============================] - 0s 87us/step - loss: 0.0439 - mean_squared_error: 0.0439
Epoch 9/10
1000/1000 [==============================] - 0s 89us/step - loss: 0.0435 - mean_squ

In [88]:
s = K.get_session()
s.run(model.weights)

[array([[0.757604 ],
        [0.6322889],
        [0.5464471]], dtype=float32)]